<a href="https://colab.research.google.com/github/gethgle/india_coved19/blob/main/INDIA_COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import requests
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster 
import pickle 
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Classroom/www.india_coved19.com

/content/drive/MyDrive/Classroom/www.india_coved19.com


In [ ]:
! git clone https://github.com/gethgle/india_coved19.git

fatal: destination path 'india_coved19' already exists and is not an empty directory.


In [ ]:
raw= requests.get("https://api.covid19india.org/raw_data.json").json()

In [ ]:
raw_data =pd.DataFrame(raw['raw_data'])
raw_data = raw_data.replace(r'^\s*$', np.nan, regex=True) 

In [ ]:
raw_data =raw_data.drop(['source1', 'source2', 'source3','backupnotes'], axis = 1) 

In [ ]:
raw_data= raw_data.dropna(how='all', subset=['agebracket','contractedfromwhichpatientsuspected',
       'currentstatus', 'dateannounced', 'detectedcity', 'detecteddistrict',
       'detectedstate', 'estimatedonsetdate', 'gender', 'nationality', 'notes','statepatientnumber',
       'statuschangedate', 'typeoftransmission'])

In [ ]:
raw_data= raw_data.dropna(how='all', subset=['detectedcity', 'detecteddistrict','detectedstate'])

In [ ]:
raw_data = raw_data[['patientnumber','contractedfromwhichpatientsuspected','statepatientnumber','dateannounced','agebracket','gender','detectedcity', 'detecteddistrict','detectedstate','currentstatus','notes','nationality','typeoftransmission','statuschangedate']]

In [ ]:
raw_data['detecteddistrict'] = raw_data.apply(lambda row: row['detectedstate'] if pd.isnull(row['detecteddistrict']) else row['detecteddistrict'],axis=1)
raw_data['detectedcity'] = raw_data.apply(lambda row: row['detecteddistrict'] if pd.isnull(row['detectedcity']) else row['detectedcity'],axis=1)  

In [ ]:
raw_data['Address'] = raw_data.detecteddistrict.str.cat(raw_data['detectedstate'],sep= (',')) 
raw_data['Address'] = raw_data.detectedcity.str.cat(raw_data['Address'],sep= (',')) 

In [ ]:
raw_data['Address'] = raw_data['Address'].str.lower() 

In [ ]:
from collections import OrderedDict
raw_data['Address'] = (raw_data['Address'].str.split(',').apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(',')) 

In [ ]:
raw_data =raw_data.drop(['detectedcity', 'detecteddistrict','detectedstate'], axis = 1) 

In [ ]:
cities =list(raw_data.Address.unique())
cities.sort()
cities = [x.lower() for x in cities] 
cities = list(dict.fromkeys(cities)) 
pd.Series(cities,name = 'Address').to_csv(r'address.csv', index = True) 

In [ ]:
lon_lat = pd.read_csv('/content/drive/MyDrive/Classroom/www.india_coved19.com/address_latlog - address.csv',index_col=0) 

In [ ]:
lon_lat= lon_lat.set_index('Address').T.to_dict('list') 

In [ ]:
#worng = list(raw_data[raw_data.city_coord.isnull()].detectedcity.unique())
#worng = ['Kalanadu','Gautam Buddh Nagar','Shrungartali','Khanyar','Eriyal','Gautam Puri','Chusot','Lakhapt','Hosayellapur','Shahid Bhagat Singh Nagar','Purasaivakkam','Bandipore','Natipora','Qutbullapur','Prakulam','Nizamuddin area','Tollygugne','Karampudi','Kunkala Marru','Hindpidi','S.P.S. Nellore','Annaimalai','Karunkalpalayam','Shahgunj','Golekbana','Kiramani Colony','South Salmara Mancachar','Y.S.R.','bomikhal','Bhiwara','Bomikhal','Hatras','Chota Udaipur','Bhimatangi, Bhubaneswar','Dera Bassi, Mohali','Bomikhal, Bhubaneswar']
#right=  ['Kalanad','Greater Noida','Shringartali','Khaniyar','Kasaragod','south delhi','leh','Lakhpat','Dharwad','Shaheed Bhagat Singh Nagar','Purasawalkam','Bandipora','Srinagar','Quthbullapur','Prakkulam','Nizamuddin','Tollygunge','Karempudi','Kunkalamarru','Hindpiri','Nellore','Annamalai','Erode','Shahganj','Bidar','Bidar','South Salmara Mankachar','KADAPA','Bhubaneswar','Bhilwara','Bhubaneswar','Hathras','Chhota Udaipur','Bhubaneswar','Dera Bassi','Bhubaneswar'] 
#raw_data['detectedcity']=raw_data.detectedcity.replace(to_replace = worng, value =right) 

In [ ]:
missing_city = []
for city in cities:
  if city in lon_lat:
    pass
  else:
    missing_city.append(city) 

In [ ]:
locator = Nominatim(user_agent="myGeocoder", timeout=5)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
city_dict = dict(zip(missing_city, pd.Series(missing_city).apply(locator.geocode).apply(lambda x: [x.latitude, x.longitude] if x else None )))
citys_location_dict = {**city_dict,**lon_lat} 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
'''#locations = pickle.load(open( "/content/drive/My Drive/Classroom/location.p", "rb" )) 
cities =list(raw_data.detectedcity.unique())
cities.sort()
cities = [x.lower() for x in cities] 
cities = list(dict.fromkeys(cities)) 
import os
from geopy.extra.rate_limiter import RateLimiter
if not os.path.isfile('locations_dict.pickle'):
  locator = Nominatim(user_agent="myGeocoder", timeout=5)
  geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
  city_dicts = dict(zip(cities, pd.Series(cities).apply(locator.geocode).apply(lambda x: [x.latitude, x.longitude] if x else None )))
  pickle_out = open("locations_dict.pickle","wb") 
  pickle.dump(city_dicts,pickle_out)
  pickle_out.close() 
else:
  pickle_in = open("locations_dict.pickle","rb")
  citys_dict= pickle.load(pickle_in) 
  pickle_in.close() 
  missing_city = []
  for city in cities:
    if city in citys_dict:
      pass
    else:
      missing_city.append(city)    
  locator = Nominatim(user_agent="myGeocoder", timeout=5)
  geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
  city_dict = dict(zip(missing_city, pd.Series(missing_city).apply(locator.geocode).apply(lambda x: [x.latitude, x.longitude] if x else None )))
  citys_location_dict = {**city_dict,**citys_dict}
  pickle_out = open("locations_dict.pickle","wb")
  pickle.dump(citys_location_dict,pickle_out) 
  pickle_out.close() '''

'#locations = pickle.load(open( "/content/drive/My Drive/Classroom/location.p", "rb" )) \ncities =list(raw_data.detectedcity.unique())\ncities.sort()\ncities = [x.lower() for x in cities] \ncities = list(dict.fromkeys(cities)) \nimport os\nfrom geopy.extra.rate_limiter import RateLimiter\nif not os.path.isfile(\'locations_dict.pickle\'):\n  locator = Nominatim(user_agent="myGeocoder", timeout=5)\n  geocode = RateLimiter(locator.geocode, min_delay_seconds=1)\n  city_dicts = dict(zip(cities, pd.Series(cities).apply(locator.geocode).apply(lambda x: [x.latitude, x.longitude] if x else None )))\n  pickle_out = open("locations_dict.pickle","wb") \n  pickle.dump(city_dicts,pickle_out)\n  pickle_out.close() \nelse:\n  pickle_in = open("locations_dict.pickle","rb")\n  citys_dict= pickle.load(pickle_in) \n  pickle_in.close() \n  missing_city = []\n  for city in cities:\n    if city in citys_dict:\n      pass\n    else:\n      missing_city.append(city)    \n  locator = Nominatim(user_agent="myGeo

In [ ]:
#pickle_in = open("locations_dict.pickle","rb")
#citys_dict= pickle.load(pickle_in) 
#pickle_in.close()

In [ ]:
raw_data['city_coord'] = raw_data['Address'].str.lower().map(citys_location_dict) 
raw_data[['latitude','longitude']] = raw_data.city_coord.apply(pd.Series) 

In [ ]:
raw_data[raw_data.city_coord.isnull()]

,patientnumber,contractedfromwhichpatientsuspected,statepatientnumber,dateannounced,agebracket,gender,currentstatus,notes,nationality,typeoftransmission,statuschangedate,Address,city_coord,latitude,longitude


In [ ]:
import folium
from folium import Map, Marker, GeoJson, LayerControl
from folium.plugins import FastMarkerCluster, MarkerCluster
map_osm = folium.Map(location=[21.7679,78.8718],zoom_start=6)
mc = MarkerCluster(name="Marker Cluster")
for index, row in raw_data.iterrows():
  if (row["currentstatus"] == "Hospitalized"):
    popup_text = "<b>Pid:</b> {}<br><b>Test Result Date:</b>{}<br><b>Age:</b> {}<br><b>Gender:</b>{}<br><b>Location:</b> {}<br><b>Notes:</b> {}<br><b>Status:</b> {}<br>".format(index+1, row["dateannounced"], row["agebracket"],row['gender'],row['Address'],row['notes'],row['currentstatus'])
    popup = folium.Popup(popup_text,max_width= 3000) 
    marker_not_active = folium.CircleMarker(location=(row["latitude"],row["longitude"]),radius= 10,color='blue',fill=True,popup=popup).add_to(mc)
  elif (row["currentstatus"]=="Recovered"):

    popup_text ="<b>Pid:</b> {}<br><b>Test Result Date:</b>{}<br><b>Age:</b> {}<br><b>Gender:</b>{}<br><b>Location:</b> {}<br><b>Notes:</b> {}<br><b>Status:</b> {}<br>".format(index+1, row["dateannounced"], row["agebracket"],row['gender'],row['Address'],row['notes'],row['currentstatus'])
    popup = folium.Popup(popup_text, max_width=3000)
    marker_active = folium.CircleMarker(location=(row["latitude"],row["longitude"]), radius=10,color='green',fill=True,popup=popup_text).add_to(mc)
  else:
    popup_text = "<b>Pid:</b> {}<br><b>Test Result Date:</b>{}<br><b>Age:</b> {}<br><b>Gender:</b>{}<br><b>Location:</b> {}<br><b>Notes:</b> {}<br><b>Status:</b> {}<br>".format(index+1, row["dateannounced"], row["agebracket"],row['gender'],row['Address'],row['notes'],row['currentstatus'])
    popup = folium.Popup(popup_text,max_width=3000)
    marker_active = folium.CircleMarker(location=(row["latitude"],row["longitude"]), radius=10,color='red',fill=True,popup=popup_text).add_to(mc)
mc.add_to(map_osm)
folium.LayerControl().add_to(map_osm)

map_osm.save('index.html')
map_osm